# Tutorial on how to use mescal
*mescal* is a [Brightway](https://docs.brightway.dev/en/latest/)-powered Python package that helps you to integrate Life-Cycle Assessment (LCA) in your energy system model

## Set up

In [1]:
# Import the required libraries
from mescal import *
import pandas as pd
import bw2data as bd

In [2]:
ecoinvent_version = '3.9.1'
esm_location = 'CA-QC'

In [3]:
# Set up your Brightway project
bd.projects.set_current(f'ecoinvent{ecoinvent_version}') # put the name of your brightway project here

## Your data
For mescal to understand the structure of your energy system model, you need to provide it with a set of dataframes.

### Mandatory dataframes

In [4]:
mapping = pd.read_csv(f'../dev/energyscope_data/{esm_location}/mapping_{ecoinvent_version}.csv')
unit_conversion = pd.read_csv(f'../dev/energyscope_data/{esm_location}/unit_conversion_{ecoinvent_version}.csv')
mapping_esm_flows_to_CPC = pd.read_csv(f'../dev/energyscope_data/{esm_location}/mapping_esm_flows_to_CPC.csv')
model = pd.read_csv(f'../dev/energyscope_data/{esm_location}/model.csv')

A mapping between the energy technologies and resources of the energy system model, and Life-Cycle Inventories datasets (LCI) from an LCI database (e.g., ecoinvent). The mapping should be provided in a dataframe with the following columns:
- `Name`: the name of the energy technology or resource in the energy model 
- `Type`: the type of the energy technology or resource (i.e., 'Construction', 'Operation', or 'Resource')  
- `Product`: the name of the product of the energy technology or resource in the LCI database
- `Activity`: the name of the activity of the energy technology or resource in the LCI database
- `Location`: the name of the region of the energy technology or resource in the LCI database
- `Unit`: (optional) the physical unit of the energy technology or resource in the LCI database
- `Database`: the name of the database in your brightway project

In [5]:
mapping.head()

,Name,Type,Product,Activity,Location,Database
0,AEC_OG,Operation,"hydrogen, gaseous, 20 bar","hydrogen production, gaseous, 20 bar, from AEC...",CH,ecoinvent_cutoff_3.9.1_remind_SSP2-Base_2020_w...
1,AEC_OG_PLANT,Construction,"electrolyzer, 1MWe, AEC, Balance of Plant","electrolyzer production, 1MWe, AEC, Balance of...",RER,ecoinvent_cutoff_3.9.1_remind_SSP2-Base_2020_w...
2,AEC_OG_PLANT_DECOM,Construction,"used fuel cell balance of plant, 1MWe, AEC","treatment of fuel cell balance of plant, 1MWe,...",RER,ecoinvent_cutoff_3.9.1_remind_SSP2-Base_2020_w...
3,AEC_OG_STACK,Construction,"electrolyzer, 1MWe, AEC, Stack","electrolyzer production, 1MWe, AEC, Stack",RER,ecoinvent_cutoff_3.9.1_remind_SSP2-Base_2020_w...
4,AEC_OG_STACK_DECOM,Construction,"used fuel cell stack, 1MWe, AEC","treatment of fuel cell stack, 1MWe, AEC",RER,ecoinvent_cutoff_3.9.1_remind_SSP2-Base_2020_w...


A set of unit conversion factors between the energy system model and the LCI database. The conversion factors should be provided in a dataframe with the following columns:
- `Name`: the name of the energy technology or resource in the energy model 
- `Type`: the type of the energy technology or resource (i.e., 'Construction', 'Operation', or 'Resource')
- `Value`: the numerical value of the conversion factor that will multiply the impact scores. It actually denotes the conversion from Impact / LCA unit to Impact / ESM unit. 
- `From`: the unit of the energy technology or resource to be converted
- `To`: the target unit of the energy technology or resource

In [6]:
unit_conversion.head()

,Name,Type,Value,To,From
0,ACETIC_ACID,Resource,247422.680400,kg,GWh
1,ACETONE,Resource,121654.501200,kg,GWh
2,AEC_OG,Construction,1555.555556,unit,GW
3,AEC_OG,Operation,30030.030030,kg,GWh
4,AEC_OG_PLANT,Construction,1.000000,unit,unit


A mapping between the energy model flows and CPC categories. The list of products should at least encompass the list of technologies without a CPC category within your mapping and their foreground inventory. The mapping should be provided in a dataframe with the following columns:
- `Product`: the name of the product in the LCI database
- `Description`: (optional) the description of the product
- `CPC`: the list of names of corresponding CPC categories

In [7]:
mapping_esm_flows_to_CPC.head()

,Flow,Description,CPC
0,BENZENE,Benzene,"['33100: Coke and semi-coke of coal, of lignit..."
1,BIO_DIESEL,Bio-diesel,['35491: Biodiesel']
2,CO2_A,Carbon dioxide (concentrated emissions),['34210b: Carbon dioxide and monoxide']
3,CO2_C,Carbon dioxide (captured),['34210b: Carbon dioxide and monoxide']
4,CO2_CS,Carbon dioxide (captured & stored),['34210b: Carbon dioxide and monoxide']


The input and output flows if energy technologies. It should be provided in a dataframe with the following columns:
- `Name`: the name of the energy technology in the energy model
- `Flow`: the name of the input or output flow
- `Amount`: the numerical value of the flow (negative if input, positive if output)  

In [8]:
model.head()

,Name,Flow,Amount
0,DIESEL_S,DIESEL_S,1.0
1,GASOLINE_S,GASOLINE_S,1.0
2,ELEC_S,ELEC_S,1.0
3,NG_S,NG_S,1.0
4,H2_S,H2_S,1.0


### Optional dataframes

In [5]:
technology_compositions = pd.read_csv(f'../dev/energyscope_data/{esm_location}/technology_compositions.csv')
technology_specifics = pd.read_csv(f'../dev/energyscope_data/{esm_location}/technology_specifics.csv') 
lifetime = pd.read_csv(f'../dev/energyscope_data/{esm_location}/lifetime.csv')
mapping_product_to_CPC = pd.read_csv('../dev/data/mapping_product_to_CPC.csv')
impact_abbrev = pd.read_csv('../dev/IW+/impact_abbrev.csv')
technologies_to_remove_from_layers = pd.read_csv(f'../dev/energyscope_data/{esm_location}/technologies_to_remove_from_layers.csv')
new_end_use_types = pd.read_csv(f'../dev/energyscope_data/{esm_location}/new_end_use_types.csv')
premise_changes = pd.read_csv('../dev/data/premise_change_report.csv')

A set of composition of technologies, i.e., if one technology or resource in the energy model should be represented by a combination of LCI datasets. The composition should be provided in a dataframe with the following columns:
- `Name`: the name of the main energy technology or resource in the energy model 
- `Components`: the list of names of subcomponents

In [10]:
technology_compositions.head()

,Name,Components
0,AEC_OG,"['AEC_OG_STACK', 'AEC_OG_PLANT', 'AEC_OG_STACK..."
1,ALKALINE_ELECTROLYSIS,"['ALKALINE_ELECTROLYSIS_STACK', 'ALKALINE_ELEC..."
2,AN_DIG_SI,"['AN_DIG_SI_PLANT', 'AN_DIG_SI_COGEN']"
3,ATR,"['ATR_PLANT', 'ATR_TANK']"
4,ATR_CCS,"['ATR_CCS_PLANT', 'ATR_CCS_TANK']"


A set of technologies with specific requirements. For instance, this stands for energy technologies without a construction phase, mobility technologies (if mismatch fuel in the LCI dataset), bioprocesses (if mismatch fuel in the LCI dataset). The requirements should be provided in a dataframe with the following columns:
- `Name`: the name of the energy technology in the energy model
- `Specifics`: the list of requirements  
- `Amount`: the numerical value of the requirement (if relevant)

In [11]:
technology_specifics.head()

,Name,Specifics,Amount
0,CO2_TO_DIESEL,Background search,2.0
1,CO2_TO_JETFUELS,Background search,0.0
2,CROPS_TO_JETFUELS,Background search,2.0
3,FT,Background search,3.0
4,GASIFICATION_SNG,Background search,1.0


Energy technologies lifetimes in the ESM and the LCI database. The lifetimes should be provided in a dataframe with the following columns:
- `Name`: the name of the energy technology in the energy model
- `ESM`: the numerical value of the lifetime in the energy system model (if relevant)
- `LCA`: the numerical value of the lifetime in the LCI database (if relevant)

In [12]:
lifetime.head()

,Name,ESM,LCA
0,AEC_OG,20.0,NaN
1,AEC_OG_PLANT,NaN,20.0
2,AEC_OG_STACK,NaN,7.5
3,AEC_OG_PLANT_DECOM,NaN,20.0
4,AEC_OG_STACK_DECOM,NaN,7.5


In case you have a LCI database without CPC categories (which are necessary for the double-counting check), you can provide a mapping between the products and activities in the LCI database and the CPC categories. The mapping should be provided in a dataframe with the following columns:
- `Name`: the (partial) name of the product or activity in the LCI database
- `CPC`: the number and name of the corresponding CPC category
- `Search type`: whether the `Name` entry is an exactly the name to look for, or is contained in the full name
- `Where`: whether the `Name` entry is meant for products or activities

In [13]:
mapping_product_to_CPC.head()

,Name,CPC,Search type,Where
0,amine-based silica,"35310: Organic surface active agents, except soap",equals,Product
1,biodiesel,35491: Biodiesel,contains,Product
2,biogas,"17200: Coal gas, water gas, producer gas and s...",equals,Product
3,"biomass, used as fuel",31230: Wood in chips or particles,equals,Product
4,"biomethane, from biogas upgrading, using amine...","12020: Natural gas, liquefied or in the gaseou...",equals,Product


An abbreviation scheme for the impact categories you aim to work with, to ease the readability in the ESM. The abbreviations should be provided in a dataframe with the following columns:
- `Impact_category`: the name of the impact category, expressed as a tuple following brightway convention
- `Unit`: (optional) the unit of the impact category
- `Abbrev`: the abbreviation of the impact category
- `AoP`: the area of protection of the impact category

In [14]:
impact_abbrev.head()

,Impact_category,Unit,Abbrev,AoP
0,"('IMPACT World+ Damage 2.0.1', 'Ecosystem qual...",PDF.m2.yr,CCEQL,EQ
1,"('IMPACT World+ Damage 2.0.1', 'Ecosystem qual...",PDF.m2.yr,CCEQS,EQ
2,"('IMPACT World+ Damage 2.0.1', 'Ecosystem qual...",PDF.m2.yr,CCEQLB,EQ
3,"('IMPACT World+ Damage 2.0.1', 'Ecosystem qual...",PDF.m2.yr,CCEQSB,EQ
4,"('IMPACT World+ Damage 2.0.1', 'Human health',...",DALY,CCHHL,HH


In case you want to remove some energy technologies from energy layers in the results LCI datasets to be added to the LCI database, you can provide a list of technologies to remove. The list should be provided in a dataframe with the following columns:
- `Layers`: name of the layer(s). A layer is basically an energy vector, which is an output for some energy technologies and an input for some others. 
- `Technologies`: the name of the energy technology to remove from the layer(s)
- `Comment`: (optional) a comment on the removal

In [15]:
technologies_to_remove_from_layers.head()

,Layers,Technologies,Comment
0,"['ELECTRICITY_EHV', 'ELECTRICITY_HV']","['TRAFO_HE','TRAFO_EH']",The high and extra high voltage electricity ar...
1,['ELECTRICITY_LV'],['STO_ELEC'],The storage technologies should be removed (pr...
2,"['NG_HP', 'NG_EHP']","['NG_EXP_EH', 'NG_EXP_EH_COGEN', 'NG_COMP_HE',...",The high and extra high pressure natural gas a...
3,"['H2_LP', 'H2_MP', 'H2_HP', 'H2_EHP']","['H2_COMP_HE', 'H2_COMP_MH', 'H2_COMP_LM', 'H2...",All pressure levels for hydrogen are merged in...
4,"['HEAT_HIGH_T', 'HEAT_LOW_T_DHN']",['HT_LT'],The high and low heat production at the DHN le...


If you want to reformat your end-use types (e.g., output layer) in order to better fit the LCI database, you can provide a list of new end-use types. The list should be provided in a dataframe with the following columns:
- `Name`: name of technologies for which the end-use type should be changed
- `Search type`: whether the `Name` entry is an exactly the name to look for (`equals`), is contained in the full name (`contains`), or is the beginning of the full name (`startswith`)
- `Old`: the current end-use type
- `New`: the new end-use type

In [16]:
new_end_use_types.head()

,Name,Search type,Old,New
0,BUS,startswith,MOB_PUBLIC,MOB_PUBLIC_BUS
1,SCHOOLBUS,startswith,MOB_PUBLIC,MOB_PUBLIC_SCHOOLBUS
2,COACH,startswith,MOB_PUBLIC,MOB_PUBLIC_COACH
3,TRAIN,startswith,MOB_PUBLIC,MOB_PUBLIC_TRAIN
4,CAR,startswith,MOB_PRIVATE,MOB_PRIVATE_CAR


If you are facing changes in names or reference products due to a new version of premise, and do not want to modify your mapping file, you should provide the dataframe with the following columns:
- `Product - old`: old reference product of the activity
- `Activity - old`: old name of the activity
- `Location - old`: old location of the activity
- `Product - new`: new reference product of the activity
- `Activity - new`: new name of the activity
- `Location - new`: new location of the activity

In [17]:
premise_changes.head()

,Product - old,Activity - old,Location - old,Product - new,Activity - new,Location - new
0,"hydrogen, gaseous, 25 bar","hydrogen production, auto-thermal reforming of...",RER,"hydrogen, gaseous, low pressure","hydrogen production, auto-thermal reforming",RER
1,"hydrogen, gaseous, 25 bar","hydrogen production, auto-thermal reforming of...",RER,"hydrogen, gaseous, low pressure","hydrogen production, auto-thermal reforming, w...",RER
2,"hydrogen, gaseous, 25 bar","hydrogen production, auto-thermal reforming, f...",CH,"hydrogen, gaseous, low pressure","hydrogen production, auto-thermal reforming, f...",RER
3,"hydrogen, gaseous, 25 bar","hydrogen production, auto-thermal reforming, f...",CH,"hydrogen, gaseous, low pressure","hydrogen production, auto-thermal reforming, f...",RER
4,"hydrogen, gaseous, 26 bar","hydrogen production, steam methane reforming, ...",CH,"hydrogen, gaseous, low pressure","hydrogen production, steam methane reforming, ...",RER


## Create a new database with additional CPC categories (optional)
In case you are working with a LCI database without CPC categories, you can create a new database with the CPC categories. The function `create_new_database_with_CPC_categories` takes as input the database with missing CPC categories, the name of the new database, and a mapping between the products and activities in the LCI database and the CPC categories. It creates a new database with the CPC categories. This step can take a few minutes depending on the size of the database.

In [6]:
name_premise_db = f"ecoinvent_cutoff_{ecoinvent_version}_remind_SSP2-Base_2020"
name_premise_with_CPC_db = name_premise_db + '_with_CPC'

In [7]:
premise_db = load_extract_db(name_premise_db)

Getting activity data


100%|██████████| 31782/31782 [00:00<00:00, 73432.23it/s] 


Adding exchange data to activities


100%|██████████| 1087635/1087635 [01:02<00:00, 17319.25it/s]


Filling out exchange data


100%|██████████| 31782/31782 [00:04<00:00, 6542.90it/s] 


In [8]:
create_new_database_with_CPC_categories(db=premise_db, new_db_name=name_premise_with_CPC_db, mapping_product_to_CPC=mapping_product_to_CPC)

Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:50


Title: Writing activities to SQLite3 database:
  Started: 07/03/2024 15:17:15
  Finished: 07/03/2024 15:18:05
  Total time elapsed: 00:00:50
  CPU %: 27.20
  Memory %: 29.13


## Operations on the mapping dataframe (optional)

In [9]:
premise_db_with_CPC = load_extract_db(name_premise_with_CPC_db, create_pickle=True)

### Relink the mapping dataframe with another database
In this example, we relink the current mapping, which is based on ecoinvent and some additional inventories, with a [premise](https://premise.readthedocs.io/en/latest/introduction.html) database. The function `create_complementary_database` takes as input the mapping dataframe, the database to link to, and the name of a newly created complementary database, in case some LCI datasets are missing in the LCI database we relink to. It returns a new mapping dataframe with the location column updated, and create the complementary database in the brightway project.

In [10]:
name_premise_comp_with_CPC_db = name_premise_with_CPC_db + f'_comp_{esm_location}'

In [11]:
mapping_linked_to_premise = create_complementary_database(mapping, premise_db_with_CPC, name_premise_comp_with_CPC_db, premise_changes)

Getting activity data


100%|██████████| 939/939 [00:00<00:00, 255014.99it/s]


Adding exchange data to activities


100%|██████████| 30812/30812 [00:01<00:00, 22935.48it/s]


Filling out exchange data


...

Filling out exchange data


100%|██████████| 935/935 [00:00<00:00, 1129.46it/s]


regional delivery_truck.pickle created!
No inventory in the premise database for ('LCV_BIODIESEL_B100_MD', 'Construction')
No inventory in the premise database for ('LCV_BIODIESEL_B100_MD', 'Operation')
No inventory in the premise database for ('LCV_BIODIESEL_B100_SD', 'Operation')
No inventory in the premise database for ('LCV_BIODIESEL_B100_SD', 'Construction')
No inventory in the premise database for ('LCV_BIODIESEL_B20_MD', 'Construction')
No inventory in the premise database for ('LCV_BIODIESEL_B20_MD', 'Operation')
No inventory in the premise database for ('LCV_BIODIESEL_B20_SD', 'Construction')
No inventory in the premise database for ('LCV_BIODIESEL_B20_SD', 'Operation')
No inventory in the premise database for ('LCV_CNG_MD', 'Construction')
No inventory in the premise database for ('LCV_CNG_MD', 'Operation')
No inventory in the premise database for ('LCV_CNG_SD', 'Operation')
No inventory in the premise database for ('LCV_CNG_SD', 'Construction')
No inventory in the premise da

Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 07/08/2024 14:21:22
  Finished: 07/08/2024 14:21:22
  Total time elapsed: 00:00:00
  CPU %: 7.90
  Memory %: 25.43


In [12]:
premise_comp_db = load_extract_db(name_premise_comp_with_CPC_db)

Getting activity data


100%|██████████| 84/84 [00:00<?, ?it/s]


Adding exchange data to activities


100%|██████████| 1681/1681 [00:00<00:00, 27901.94it/s]


Filling out exchange data


100%|██████████| 84/84 [00:00<00:00, 198.98it/s]


In [13]:
create_new_database_with_CPC_categories(db=premise_comp_db, new_db_name=name_premise_comp_with_CPC_db, mapping_product_to_CPC=mapping_product_to_CPC)

Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 07/08/2024 14:21:32
  Finished: 07/08/2024 14:21:32
  Total time elapsed: 00:00:00
  CPU %: 22.20
  Memory %: 8.52


In [14]:
premise_comp_db = [] # free memory

In [15]:
mapping_linked_to_premise.head()

,Name,Type,Product,Activity,Location,Database
0,AEC_OG,Operation,"hydrogen, gaseous, 20 bar","hydrogen production, gaseous, 20 bar, from AEC...",CH,ecoinvent_cutoff_3.9.1_remind_SSP2-Base_2020_w...
1,AEC_OG_PLANT,Construction,"electrolyzer, 1MWe, AEC, Balance of Plant","electrolyzer production, 1MWe, AEC, Balance of...",RER,ecoinvent_cutoff_3.9.1_remind_SSP2-Base_2020_w...
2,AEC_OG_PLANT_DECOM,Construction,"used fuel cell balance of plant, 1MWe, AEC","treatment of fuel cell balance of plant, 1MWe,...",RER,ecoinvent_cutoff_3.9.1_remind_SSP2-Base_2020_w...
3,AEC_OG_STACK,Construction,"electrolyzer, 1MWe, AEC, Stack","electrolyzer production, 1MWe, AEC, Stack",RER,ecoinvent_cutoff_3.9.1_remind_SSP2-Base_2020_w...
4,AEC_OG_STACK_DECOM,Construction,"used fuel cell stack, 1MWe, AEC","treatment of fuel cell stack, 1MWe, AEC",RER,ecoinvent_cutoff_3.9.1_remind_SSP2-Base_2020_w...


### Add or replace the location column based on a user-defined ranking
Based on a user-defined ranking, the location column of the mapping dataframe can be updated. The function `change_location_mapping_file` takes as input the mapping dataframe, the user-defined ranking, and the base database. It returns the mapping dataframe with the location column updated.

In [16]:
# create a concatenated database of all databases in the mapping dataframe (including background requirements)
base_db = concatenate_databases(list(mapping_linked_to_premise.Database.unique()))

Getting activity data


100%|██████████| 84/84 [00:00<?, ?it/s]


Adding exchange data to activities


100%|██████████| 1681/1681 [00:00<00:00, 38710.56it/s]


Filling out exchange data


100%|██████████| 84/84 [00:00<00:00, 438.37it/s]


In [15]:
# Define the user-defined ranking
if esm_location == 'CA-QC':
    my_ranking = [
        'CA-QC', # Quebec
        'CA', # Canada
        'CA-ON', # Other canadian provinces 
        'CA-AB',
        'CA-BC',
        'CA-MB',
        'CA-NB',
        'CA-NF',
        'CA-NS',
        'CA-NT',
        'CA-NU',
        'CA-PE',
        'CAZ', # Canada - Australia - New Zealand
        'RNA', # North America
        'US', # United States
        'USA', # United States
        'GLO', # Global average 
        'RoW', # Rest of the world
    ]
elif esm_location == 'CH':
    my_ranking = [
        'CH', 
        'RER', 
        'IAI Area, EU27 & EFTA',
        'NEU',
        'EUR',
        'GLO',
        'RoW'
    ]
else:
    my_ranking = [
        'GLO',
        'RoW',
    ]

In [18]:
# Update mapping dataframe
mapping_linked_to_premise = change_location_mapping_file(mapping_linked_to_premise, my_ranking, base_db, esm_location)

In [19]:
mapping_linked_to_premise.head()

,Name,Type,Product,Activity,Location,Database
0,AEC_OG,Operation,"hydrogen, gaseous, 20 bar","hydrogen production, gaseous, 20 bar, from AEC...",CH,ecoinvent_cutoff_3.9.1_remind_SSP2-Base_2020_w...
1,AEC_OG_PLANT,Construction,"electrolyzer, 1MWe, AEC, Balance of Plant","electrolyzer production, 1MWe, AEC, Balance of...",RER,ecoinvent_cutoff_3.9.1_remind_SSP2-Base_2020_w...
2,AEC_OG_PLANT_DECOM,Construction,"used fuel cell balance of plant, 1MWe, AEC","treatment of fuel cell balance of plant, 1MWe,...",RER,ecoinvent_cutoff_3.9.1_remind_SSP2-Base_2020_w...
3,AEC_OG_STACK,Construction,"electrolyzer, 1MWe, AEC, Stack","electrolyzer production, 1MWe, AEC, Stack",RER,ecoinvent_cutoff_3.9.1_remind_SSP2-Base_2020_w...
4,AEC_OG_STACK_DECOM,Construction,"used fuel cell stack, 1MWe, AEC","treatment of fuel cell stack, 1MWe, AEC",RER,ecoinvent_cutoff_3.9.1_remind_SSP2-Base_2020_w...


In [20]:
# Save the relinked mapping file
mapping_linked_to_premise.to_csv(f'../dev/energyscope_data/{esm_location}/mapping_{ecoinvent_version}_linked.csv', index=False)

## Double-counting removal

In [11]:
# To skip the previous steps
# mapping_linked_to_premise = pd.read_csv(f'../dev/energyscope_data/{esm_location}/mapping_{ecoinvent_version}_linked.csv')

In [22]:
mapping_linked_to_premise = mapping_linked_to_premise[mapping_linked_to_premise['Type'] != 'Flow']

In [23]:
new_db_name = f'energyscope_{esm_location}_2020'

In [24]:
regionalize_foregrounds = False

In [16]:
if esm_location == 'CA-QC':
    accepted_locations = ['CA-QC', 'CA']  # sufficient match within ecoinvent
elif esm_location == 'CH':
    accepted_locations = ['CH']  # sufficient match within ecoinvent
else:
    accepted_locations = ['GLO', 'RoW']  # sufficient match within ecoinvent

In [13]:
base_db = concatenate_databases(list(mapping_linked_to_premise.Database.unique()))

Getting activity data


100%|██████████| 84/84 [00:00<00:00, 10473.91it/s]


Adding exchange data to activities


100%|██████████| 1681/1681 [00:00<00:00, 11605.60it/s]


Filling out exchange data


100%|██████████| 84/84 [00:00<00:00, 276.21it/s]


In [27]:
mapping_linked_to_premise_new_code = create_esm_database(
    mapping=mapping_linked_to_premise,
    model=model,
    tech_specifics=technology_specifics,
    technology_compositions=technology_compositions,
    mapping_esm_flows_to_CPC_cat=mapping_esm_flows_to_CPC,
    main_database=base_db,
    esm_db_name=new_db_name,
    regionalize_foregrounds=regionalize_foregrounds,
    accepted_locations=accepted_locations,
    target_region=esm_location,
    locations_ranking=my_ranking
)

AEC_OG
ALKALINE_ELECTROLYSIS
AL_MAKING
AL_MAKING_HR
AN_DIG
AN_DIG_SI
ATR
ATR_CCS
BIOGAS_ATR
BIOGAS_ATR_CCS
BIOGAS_BIOMETHANE
BIOGAS_SMR
BIOGAS_SMR_CCS
BIOMASS_ETHANOL
BIOMASS_GAS_EF_H2
BIOMASS_GAS_EF_H2_CCS
BIOMASS_GAS_FB_H2
BIOMASS_GAS_FB_H2_CCS
BUS_BIODIESEL_B100_SD
BUS_BIODIESEL_B20_SD
BUS_CNG_SD
BUS_DIESEL_SD
BUS_EV_SD
BUS_FC_CH4_SD
BUS_FC_H2_SD
BUS_HY_DIESEL_SD
BUS_PROPANE_SD
CAR_BIODIESEL_B100_ELD
CAR_BIODIESEL_B100_LD
CAR_BIODIESEL_B100_MD
CAR_BIODIESEL_B100_SD
CAR_BIODIESEL_B20_ELD
CAR_BIODIESEL_B20_LD
CAR_BIODIESEL_B20_MD
CAR_BIODIESEL_B20_SD
CAR_CNG_ELD
CAR_CNG_LD
CAR_CNG_MD
CAR_CNG_SD
CAR_DIESEL_ELD
CAR_DIESEL_LD
CAR_DIESEL_MD
CAR_DIESEL_SD
CAR_ETOH_E10_ELD
CAR_ETOH_E10_LD
CAR_ETOH_E10_MD
CAR_ETOH_E10_SD
CAR_ETOH_E85_ELD
CAR_ETOH_E85_LD
CAR_ETOH_E85_MD
CAR_ETOH_E85_SD
CAR_EV_ELD
CAR_EV_LD
CAR_EV_MD
CAR_EV_SD
CAR_FC_CH4_ELD
CAR_FC_CH4_LD
CAR_FC_CH4_MD
CAR_FC_CH4_SD
CAR_FC_H2_ELD
CAR_FC_H2_LD
CAR_FC_H2_MD
CAR_FC_H2_SD
CAR_GASOLINE_ELD
CAR_GASOLINE_LD
CAR_GASOLINE_MD
CAR_GASOLI

Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:02


Title: Writing activities to SQLite3 database:
  Started: 07/08/2024 14:23:44
  Finished: 07/08/2024 14:23:46
  Total time elapsed: 00:00:02
  CPU %: 25.00
  Memory %: 17.17


## Computing the LCA metrics 

In [28]:
esm_db = load_extract_db(new_db_name)

Getting activity data


100%|██████████| 1433/1433 [00:00<00:00, 153229.77it/s]


Adding exchange data to activities


100%|██████████| 36752/36752 [00:01<00:00, 27919.11it/s]


Filling out exchange data


100%|██████████| 1433/1433 [00:02<00:00, 494.61it/s] 


In [29]:
R_long = compute_impact_scores(
    esm_db=esm_db,
    mapping=mapping_linked_to_premise_new_code,
    technology_compositions=technology_compositions,
    methods=['IMPACT World+ Midpoint 2.0.1', 'IMPACT World+ Damage 2.0.1', 'IMPACT World+ Footprint 2.0.1'],
    unit_conversion=unit_conversion,
    lifetime=lifetime
)

In [30]:
R_long.head()

,Impact_category,New_code,Value,Name,Type
0,"(IMPACT World+ Damage 2.0.1, Ecosystem quality...",07rfjm0a05zb09gb086nb1pbtpfm468f,3.019423e+09,AFC,Construction
1,"(IMPACT World+ Damage 2.0.1, Ecosystem quality...",07rfjm0a05zb09gb086nb1pbtpfm468f,1.015762e+09,AFC,Construction
2,"(IMPACT World+ Damage 2.0.1, Human health, Cli...",07rfjm0a05zb09gb086nb1pbtpfm468f,1.374035e+04,AFC,Construction
3,"(IMPACT World+ Damage 2.0.1, Human health, Cli...",07rfjm0a05zb09gb086nb1pbtpfm468f,4.694294e+03,AFC,Construction
4,"(IMPACT World+ Midpoint 2.0.1, Midpoint, Clima...",07rfjm0a05zb09gb086nb1pbtpfm468f,5.281448e+09,AFC,Construction


In [31]:
R_long.to_csv(f'results/impact_scores_{ecoinvent_version}.csv', index=False)

## Convert the results in an AMPL format

In [7]:
# To skip the previous steps
# R_long = pd.read_csv(f'results/impact_scores_{ecoinvent_version}.csv')

In [8]:
refactor = 1e-2
lcia_method = 'IMPACT World+ Damage 2.0.1 - Total only'

### Create the .dat file

In [34]:
normalize_lca_metrics(
    R=R_long,
    f_norm=1e6,
    mip_gap=1e-6,
    refactor=refactor,
    lcia_method=lcia_method,
    impact_abbrev=impact_abbrev
)

### Create the .mod file

In [35]:
gen_lcia_obj(
    lcia_method=lcia_method,
    refactor=refactor,
    impact_abbrev=impact_abbrev
)

## Integrate the ESM results back in the LCI database

In [9]:
annual_prod = pd.read_csv(f'../dev/energyscope_data/{esm_location}/results_ES.csv') # put the path to your ESM results here

In [17]:
create_new_database_with_esm_results(
    mapping=mapping_linked_to_premise,
    model=model,
    esm_location=esm_location,
    esm_results=annual_prod,
    unit_conversion=unit_conversion,
    db=base_db,
    locations_ranking=my_ranking,
    accepted_locations=accepted_locations,
    new_db_name=f'ecoinvent_cutoff_{ecoinvent_version}_remind_SSP2-Base_2020_ESM',
    technology_compositions=technology_compositions,
    tech_specifics=technology_specifics,
    mapping_esm_flows_to_CPC_cat=mapping_esm_flows_to_CPC,
    tech_to_remove_layers=technologies_to_remove_from_layers,
    new_end_use_types=new_end_use_types,
)

The technology DHN_RENOVATION is not in the mapping file. It cannot be considered in the result LCI dataset.


ValueError: The unit conversion factor between kilometer and ton kilometer for transport, freight, lorry, unspecified is not in the unit conversion file.